# Setup for Multiple Parameter Simulations

This notebook shows how to set up multiple simulation runs for systematic parameter studies. 

## 1) Generate directories for parameter scan

Start by importing some modules:

In [ ]:
import sys
import os
import numpy as np
import tifffile as tif

Now define which input parameter should be varied. Possible keys are *memory*, *aperture*, *branching_angle*, 
*bifurcation_angle*,*branchingprob*,*bifurcationprob*,*deathprob* and *reactivationprob*.

In [ ]:
var_key = "memory";

The range of values that it should take are set in this numpy array:

In [ ]:
var_range = [1,2,3,4,5];

Now set and create the directory for this simulation if it does not yet exist. This notebook needs to run from the base directory of the framework:

In [ ]:
home = os.getcwd();
basedir = "Examples/Osteocytes"
simdir = os.path.join(home,basedir,var_key);

if not os.path.exists(simdir):
        os.makedirs(simdir)

Next, create one subdirectoy for each value of the parameter:

In [ ]:
for val in var_range:
    new_path = basedir + "/" + str(var_key) + "/" + str(val);
    if not os.path.exists(new_path):
        os.makedirs(new_path)

This cell creates subdirectories for each parameter value corresponding to the number of identical runs:

In [ ]:
runs = 10

for v in var_range:
    for r in range(runs):
        save_dir = os.path.join(home,simdir,str(v),str(r),);
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)

***
## 2) Copy the parameter files to the new simulation directories

Make copies of all configuration files with the standard parameters for each parameter configuration.

Then, in the last line, `sed` is used to actually set the varying parameter to the predefined values in each directory:

In [ ]:
for val in var_range:
    new_path = basedir + "/" + str(var_key) + "/" + str(val);
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    os.system("cp "+basedir+"/growth_parameters.csv "+new_path)
    os.system("cp "+basedir+"/starting_positions_ocy.csv "+new_path)
    os.system("cp "+basedir+"/structured_image_dir.csv "+new_path)
    os.system("cp "+basedir+"/internal_parameters.csv "+new_path)
    os.system("sed -i \'s/"+str(var_key)+".*/"+str(var_key)+","+str(val)+"/g\' "+str(new_path)+"/internal_parameters.csv")


***

The following cells only need to be run if new starting positions and environments should be used, or if slurm job files should be created. 

***

## 3) Define new starting positions (optional)
Run this cell if you want to create a new set of starting positions and directions. Otherwise, the existing file will be used and copied to the new simulation directories.

Here, we create the cell array for the Osteocyte Network scenario:

In [ ]:
cells = np.array([[128,64,128],[128,128,128],[128,192,128],
                  [64,96,128], [64,160,128],
                  [192,96,128],[192,160,128],
                  [128,96,96], [128,160,96],
                  [64,64,96],  [64,128,96],  [64,192,96],
                  [192,64,96], [192,128,96], [192,192,96],
                  [128,96,160],[128,160,160],
                  [64,64,160], [64,128,160], [64,192,160],
                  [192,64,160],[192,128,160],[192,192,160]]);

N_cells = cells.shape[0]

# offset of cell surface from cell center
w = 7
l = 12
h = 4

# number of outgrowing processes per cell
N = 20

# equally distributed angles
idx = np.arange(0, N, dtype=float) + 0.5;
phi = np.arccos(1 - 2*idx/N);
theta = np.pi * idx * (1 + 5**0.5);

# azimuth and polar angles of initial direction
az = np.mod(np.rad2deg(phi),180);
pol = np.mod(np.rad2deg(theta),360);

# positions on cell surface relative to center
x = w * np.cos(theta) * np.sin(phi);
y = l * np.sin(theta) * np.sin(phi);
z = h * np.cos(phi)

# generate column vectors for positions of all cells
xc = np.squeeze(np.repeat(cells[:,0],N) + np.tile(x,(1,N_cells)));
yc = np.squeeze(np.repeat(cells[:,1],N) + np.tile(y,(1,N_cells)));
zc = np.squeeze(np.repeat(cells[:,2],N) + np.tile(z,(1,N_cells)));

azc = np.squeeze(np.tile(az,(1,N_cells)));
polc = np.squeeze(np.tile(pol,(1,N_cells)));

# index column to identify processes from the same cell
idc = np.repeat(np.arange(1, N_cells+1, dtype=float),N);

# save starting positions to text file
np.savetxt(os.path.join(basedir,'starting_positions_ocy.csv'),np.column_stack((xc,yc,zc,azc,polc,idc)), fmt='%.2f', delimiter=',', header='Starting Positions\nX,Y,Z,Az,Pol,CN')

# now copy again into all the subdirectories to get the new file:
for val in var_range:
    new_path = basedir + "/" + str(var_key) + "/" + str(val);
    os.system("cp "+basedir+"/starting_positions_ocy.csv "+new_path)


***
## 4) Create the structured grid for the environment (optional)

Run this cell to re-create the structured images and feature maps, common to all simulation runs. Otherwise, the already existing images will be used.

In this example, we create an anisotropic bone matrix for the Osteocyte Network scenario:

In [ ]:
import Initialize_Network_Growing as ing
import Start_Conditions.StructField_Maker as StructField_Maker# Path to this notebook

path_structured_images = os.path.join(basedir,'structured_image_dir.csv');

# generate and save empty volume 
fe = np.zeros((256,256,256));
tif.imsave(os.path.join(basedir,'empty.tif'),np.float32(fe))

# create 256³ structured image procedurally
f = np.zeros((4,4));

# some collagen fibers
f[0:2,0:2] = 300;

# tile to fill volume
f2 = np.tile(f, [256,64,64]);

# introduce some lamellae with perpendicular fibers
f2[64:256:64,:,::16] = 1;
f2[64:256:64,::16,:] = 1;

# save at the appropriate position
tif.imsave(os.path.join(basedir,'fiberz.tif'),np.float32(f2))

# Generate feature maps from image data of growth environment
features = ing.StructField_Maker.structured_field(path_structured_images, home, sigma_divd=2, sigma_divt1=2, sigma_divt2=2)

***
## 5) Create slurm job file for each parameter value (optional)

If you run this cell, a slurm .job file will be created that defines a job array for each parameter value, corresponding to the number of runs defined above. 

This example is for the Julia HPC cluster at JMU. Settings for partition, number of cores, starting the environment etc. may have to be adapted to your cluster.

From the slurm head node, the job can then be submitted by `sbatch jobfile.job`.

The simulations can also be run individually by calling:

`python run_growth.py /Examples/Osteocytes/$var_key/$var_value $run`

In [ ]:
# Path and conda environment on the compute nodes (may differ from where this notebook runs)
slurmhome = "/home/phk57mf/pythrahyper_net/";
env = "pythra-mayavi"

for v in var_range:

    job_file = os.path.join(home,simdir,str(v),var_key+"_"+str(v)+".job")

    with open(job_file,'w') as f:

            f.writelines("#!/bin/bash\n")
            f.writelines("#SBATCH --job-name=OCY_"+str(var_key)+"_"+str(v)+"_%a_"+".job\n")
            f.writelines("#SBATCH --output=.out"+"/OCY_"+str(var_key)+"_"+str(v)+"_%a.out\n")
            f.writelines("#SBATCH --error=.out"+"/OCY_"+str(var_key)+"_"+str(v)+"_%a.err\n")
            f.writelines("#SBATCH --time=2-00:00\n")
            f.writelines("#SBATCH --mem=16000\n")
            f.writelines("#SBATCH --partition=standard\n")
            f.writelines("#SBATCH -c 8\n")
            f.writelines("#SBATCH -n 1\n")
            f.writelines("#SBATCH --array=0-"+str(runs-1)+"\n")
            f.writelines("source activate "+env+"\n")
            f.writelines("cd "+slurmhome+"\n")
            f.writelines("python run_growth.py /"+os.path.join(basedir,var_key,str(v))+"/ $SLURM_ARRAY_TASK_ID\n")
            
